# 在 AlphaMed 环境中使用横向联邦算法训练自己的模型

## 横向联邦算法调度器简介

AlphaMed 平台当前共有 4 个内置 FedAvg 家族算法调度器：
- `FedAvgScheduler` 对应 MiniBatch SGD 版本的 FedAvg 算法，也是不考虑加密情况下使用最普遍的版本。
- `FedSGDScheduler` 对应 WholeBatch SGD 版本的 FedAvg 算法，训练速度非常慢，效果也一般，一般仅用于研究。`FedAvgScheduler` 的子类。
- `DPFedAvgScheduler` 使用差分隐私保护本地模型参数更新的 FedAvg 算法，训练速度与 MiniBatch SGD 版本的 FedAvg 算法相当，但会有一定的精度损失，且需要设置合适的超参数值。`FedAvgScheduler` 的子类。
- `SecureFedAvgScheduler` 使用安全聚合算法保护本地模型参数更新的 FedAvg 算法，训练速度较慢，优点是不会损失精度。`FedAvgScheduler` 的子类。

示例中使用[第 2 节](2.%20%E5%9C%A8%20AlphaMed%20%E5%B9%B3%E5%8F%B0%E4%B8%8A%E8%BF%90%E8%A1%8C%E6%99%AE%E9%80%9A%E6%A8%A1%E5%9E%8B.ipynb)中定义的 `ConvNet` 模型网络。先展示如何使用 `FedAvgScheduler` 训练，然后在此基础上介绍 `FedSGDScheduler`、`DPFedAvgScheduler`、`SecureFedAvgScheduler` 调度器。

### `FedAvgScheduler` 介绍

`FedAvgScheduler` 是个虚拟基础类，要使用 `FedAvgScheduler` 训练 `ConvNet`，需要继承实现 `FedAvgScheduler` 中定义的接口。下面先简单介绍 `FedAvgScheduler` 中指定的接口，然后设计示例实现这些接口。

In [2]:
from abc import abstractmethod
from typing import Dict

import torch
from torch.nn import Module
from torch.optim import Optimizer
from torch.utils.data import DataLoader

In [2]:
# 模型相关接口

@abstractmethod
def build_model(self) -> Module:
    """返回模型实例，本示例中返回 `ConvNet` 实例.

    此接口必须实现。
    """

@abstractmethod
def build_optimizer(self, model: Module) -> Optimizer:
    """返回优化器实例，用于本地训练时的参数更新。

    参数说明:
        model:
            训练使用的模型对象，由框架传入。

    此接口必须实现。
    """

In [3]:
# 数据集相关接口

@abstractmethod
def build_train_dataloader(self) -> DataLoader:
    """返回一个 PyTorch DataLoader 的实例，模型在本地训练过程中通过此实例获取训练集数据用于训练。

    此接口必须实现。
    """

def build_validation_dataloader(self) -> DataLoader:
    """返回一个 PyTorch DataLoader 的实例，模型在本地训练过程中通过此实例获取验证集数据用于训练。

    此接口为可选择实现接口，如果训练时需要使用验证集数据就实现这个接口，否则可以忽略。
    """

@abstractmethod
def build_test_dataloader(self) -> DataLoader:
    """返回一个 PyTorch DataLoader 的实例，发起方在参数聚合后通过此实例获取测试集数据用于测试模型训练效果。

    此接口必须实现。
    """

In [4]:
# 聚合参数相关接口

def state_dict(self) -> Dict[str, torch.Tensor]:
    """此接口返回需要参与参数聚合的参数字典。

    此接口为可选择实现接口，默认调用本地模型的 state_dict() 方法返回模型的全部参数。
    如果需要定制参数更新方式，比如：锁定一部分模型参数只更新局部参数，或者聚合时包含优化器的参数，
    """

def load_state_dict(self, state_dict: Dict[str, torch.Tensor]):
    """state_dict 接口的逆操作，用于本地模型加载更新参数。

    参数说明:
        state_dict:
            更新使用的新模型参数。

    此接口为可选择实现接口，默认调用本地模型的 load_state_dict() 更新模型参数。
    如果需要定制参数更新方式，则需要自行修改实现逻辑。
    """

In [5]:
# 训练相关接口

@abstractmethod
def train_an_epoch(self):
    """模型本地训练时，训练一个 epoch 的代码逻辑。

    由于不同模型不同场景下的具体训练方式千差万别，所以需要算法工程师自行提供训练逻辑。

    此接口必须实现。
    """

@abstractmethod
def run_test(self):
    """参数聚合后完成后，执行一次测试的代码逻辑。

    由于不同模型不同场景下的具体测试方式千差万别，所以需要算法工程师自行提供测试逻辑。

    此接口必须实现。
    """

def is_task_finished(self) -> bool:
    """判断训练过程是否结束。

    此接口为可选择实现，默认情况下判断是否达到了设置的 max_round 值，达到了就结束。
    如果有特殊需要，比如：通过验证集和早停技术避免过拟合时，需要能提前终止训练，则需要重新实现此接口逻辑。
    """

In [6]:
# 其它接口

def validate_context(self):
    """训练开始前验证运行环境，如果发现异常可以提前结束，发送消息通知前台干预。

    此接口为可选择实现，默认只检查模型实例和优化器实例是否成功加载。
    """

接口介绍完毕。

接下来定义 `DemoAvg(FedAvgScheduler)` 演示如何实现上述接口，完成横向联邦 FedAvg 训练 `ConvNet`。`DemoAvg` 代码及注释如下。

In [7]:
from time import time

import torch
import torch.nn.functional as F
import torchvision
from cnn_net import ConvNet
from torch import nn, optim
from torch.utils.data import DataLoader

from alphafed.fed_avg import FedAvgScheduler


class DemoAvg(FedAvgScheduler):

    def __init__(self,
                 max_rounds: int = 0,
                 merge_epochs: int = 1,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0,
                 involve_aggregator: bool = False,
                 batch_size: int = 128,
                 learning_rate: float = 0.01,
                 momentum: float = 0.9) -> None:
        """初始化参数说明.

        以下为 `FedAvgScheduler` 父类定义的初始化参数。
        max_rounds:
            训练多少轮。
        merge_epochs:
            每次参数聚合前，在本地训练几个 epoch。
        calculation_timeout:
            本地训练超时时间。
        log_rounds:
            每隔几轮训练执行一次测试，评估记录当前训练效果。
        involve_aggregator:
            聚合方是否也使用本地数据参与训练，默认不参与，只负责聚合。

        以下 `DemoAvg` 自定义的扩展初始化参数。
        batch_size、learning_rate、momentum
            训练参数。
        """
        super().__init__(max_rounds=max_rounds,
                         merge_epochs=merge_epochs,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds,
                         involve_aggregator=involve_aggregator)
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.momentum = momentum

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.seed = 42
        torch.manual_seed(self.seed)

    def build_model(self) -> nn.Module:  # 返回模型实例
        model = ConvNet()
        return model.to(self.device)

    def build_optimizer(self, model: nn.Module) -> optim.Optimizer:  # 返回优化器实例
        return optim.SGD(model.parameters(),
                         lr=self.learning_rate,
                         momentum=self.momentum)

    # 网络下载 MNIST 训练数据用于训练，如果是实际使用场景，需要开发者自行处理数据加载
    def build_train_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=True
        )

    # 本示例不使用验证集数据，所以不需要实现 build_validation_dataloader

    # 网络下载 MNIST 测试数据用于测试，如果是实际使用场景，需要开发者自行处理数据加载
    def build_test_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=False
        )

    def train_an_epoch(self) -> None:  # 执行一个 epoch 训练的逻辑，与本地训练模型时的代码相同
        self.model.train()
        for data, labels in self.train_loader:
            data, labels = data.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.nll_loss(output, labels)
            loss.backward()
            self.optimizer.step()

    def run_test(self):  # 执行一次测试的逻辑，与本地模型测时的代码相同
        self.model.eval()
        start = time()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, labels in self.test_loader:
                data, labels = data.to(self.device), labels.to(self.device)
                output: torch.Tensor = self.model(data)
                test_loss += F.nll_loss(output, labels, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(labels.view_as(pred)).sum().item()

        end = time()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy

        # 记录 TensorBoard 日志，self.current_round 的值为“当前是训练的第几轮”
        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

使用 `FedAvgScheduler` 训练 `ConvNet` 的调度器设计完毕，接下来演示一下如何调试和运行。

### 使用 `FedAvgScheduler` 模拟训练模型

示例使用 MNIST 数据集模拟训练[第 2 节](2.%20%E5%9C%A8%20AlphaMed%20%E5%B9%B3%E5%8F%B0%E4%B8%8A%E8%BF%90%E8%A1%8C%E6%99%AE%E9%80%9A%E6%A8%A1%E5%9E%8B.ipynb)中设计的 `ConvNet` 模型。先介绍模拟调试环节，假设此时还不确定 `DemoAvg` 的代码是否正确，能不能跑起来。可以这样做调试：
1. 将 `DemoAvg` 的代码复制粘贴到 Notebook 的一个 Cell 单元中，运行一次完成加载。加载时如果存在错误，Notebook 会显示异常信息。（可以故意修改代码制造语法错误，模拟体验这种情况。）
2. 加载成功之后需要在模拟环境中模拟运行一下代码。由于模拟环境不能实际执行向其它节点分发任务的操作（只能访问本地资源），所以需要多开几个 Notebook 脚本文件，每一个文件模拟一个节点。每个 Notebook 脚本文件都需要执行 step 1 加载 `DemoAvg`。
3. 参考下面的代码分别实例化聚合方和参与方，通过 `with mock_context():` 在模拟环境中调用开始计算的接口。（实际运行时任务管理器会自动完成实例化和调用。）运行中产生的文件数据默认会保存在“{NODE_ID}/{TASK_ID}”目录下，其中 NODE_ID 是测试脚本启动模拟环境时设置的 `id` 值，TASK_ID 是测试脚本启动调度器时传入的 `task_id` 值。

> 1. 有关模拟环境的详细信息将在后续章节补充介绍，当前不需要深究，不会影响对算法调度器的理解。
> 2. 不能在一个 Notebook 脚本文件中模拟多个节点是因为受到技术上的制约，Notebook 环境不支持一个 Notebook 脚本文件中的多个 cell 并行，只能挨个串行。而联邦学习是并行运算，所以不能使用多个 cell 模拟多个节点。

In [ ]:
from alphafed import mock_context

# 聚合方的模拟启动脚本
scheduler = DemoAvg(
    max_rounds=3,  # 受本地资源限制，运行速度可能会很慢，调试时不建议设置太高
    log_rounds=1,
    calculation_timeout=1800  # 受本地资源限制，运行速度可能会很慢，设置太低容易导致超时
)

task_id = 'cbb3ffd0-838c-41ca-a41a-7c11cae29181'  # 设置一个假想 ID
# 算法实际运行时会从任务管理器获取任务参与节点的 Node ID 列表，但是在模拟环境不能通过
# 访问实际接口获得这个信息，所以需要通过 nodes 参数将这个列表配置在模拟环境中。
aggregator_id = '1bb9feba-7b53-455b-b127-0eb19ffc9d3f'  # 设置一个假想 ID
col_id_1 = '663ad4b0-b617-409f-8bc9-3682b30f7f30'  # 设置一个假想 ID
col_id_2 = '0fc1a571-2920-47bf-9e4e-b4edb7fa2caa'  # 设置一个假想 ID
with mock_context(id=aggregator_id, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    scheduler._run(id=aggregator_id, task_id=task_id, is_initiator=True)


# 参与方的模拟启动脚本，需要复制到另一个 Notebook 脚本文件中执行
# scheduler 实例和聚合方的一模一样
scheduler = DemoAvg(max_rounds=3, log_rounds=1, calculation_timeout=1800)

task_id = 'cbb3ffd0-838c-41ca-a41a-7c11cae29181'  # 任务 ID 必须与聚合方一致
aggregator_id = '1bb9feba-7b53-455b-b127-0eb19ffc9d3f'  # 必须与聚合方配置的一致
col_id_1 = '663ad4b0-b617-409f-8bc9-3682b30f7f30'  # 必须与聚合方配置的一致
col_id_2 = '0fc1a571-2920-47bf-9e4e-b4edb7fa2caa'  # 必须与聚合方配置的一致
with mock_context(id=col_id_1, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    scheduler._run(id=col_id_1, task_id=task_id)

# 另一个参与方的模拟启动脚本，需要复制到另一个 Notebook 脚本文件中执行
# scheduler 实例和聚合方的一模一样
scheduler = DemoAvg(max_rounds=3, log_rounds=1, calculation_timeout=1800)

task_id = 'cbb3ffd0-838c-41ca-a41a-7c11cae29181'  # 任务 ID 必须与聚合方一致
aggregator_id = '1bb9feba-7b53-455b-b127-0eb19ffc9d3f'  # 必须与聚合方配置的一致
col_id_1 = '663ad4b0-b617-409f-8bc9-3682b30f7f30'  # 必须与聚合方配置的一致
col_id_2 = '0fc1a571-2920-47bf-9e4e-b4edb7fa2caa'  # 必须与聚合方配置的一致
with mock_context(id=col_id_2, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    scheduler._run(id=col_id_2, task_id=task_id)

整理好的[聚合方脚本](res/3_aggregator.ipynb)、[参与方-1 脚本](res/3_collaborator_1.ipynb)、[参与方-2 脚本](res/3_collaborator_2.ipynb)均可以直接运行。

在模拟环境调试运行成功之后，对聚合方的启动脚本稍作修改，就可以在正式任务中使用了。参考下面的代码示例和注释说明：

In [ ]:
# 聚合方的模拟启动脚本
scheduler = DemoAvg(max_rounds=5, log_rounds=1)

# 这些模拟调试的代码不需要了
# aggregator_id = '1bb9feba-7b53-455b-b127-0eb19ffc9d3f'  # 随便设置一个
# task_id = 'cbb3ffd0-838c-41ca-a41a-7c11cae29181'  # 随便设置一个
# with mock_context(id=aggregator_id):  # 在模拟调试环境中运行
#     scheduler._run(id=aggregator_id, task_id=task_id, is_initiator=True)

# 将调度器代码及相关资源上传至运行环境，等待 Playground 启动计算任务
scheduler.submit(task_id='YOUR_TASK_ID')

执行横向联邦学习任务需要登录 [AlphaMed Playground 客户端](https://alphamed.ssplabs.com/)，[这里](../fed_avg/README.ipynb)有创建横向联邦学习任务的详细说明，请按照说明中的步骤运行示例程序。

### `FedSGDScheduler` 介绍

`FedSGDScheduler` 继承了 `FedAvgScheduler`, 是其子类。设计 `FedSGDScheduler` 与设计 `FedAvgScheduler` 相比，仅有初始化参数和 `build_train_dataloader` 接口实现有差异，其它地方完全一致。这是因为 FedSGD 算法要求：每轮训练必须包括所有参与方、本地训练时 `batch_size` 必须设置为全部训练样本数量、每一轮本地训练只能执行一个 epoch，所以与此相关的参数（`batch_size` 和 `merge_epochs`）都不允许自定义。

***再次提醒，FedSGD 算法一般仅用于研究，不适合应用于现实场景中。***

In [ ]:
from time import time

import torch
import torch.nn.functional as F
import torchvision
from cnn_net import ConvNet
from torch import nn, optim
from torch.utils.data import DataLoader

from alphafed.fed_avg import FedSGDScheduler


class DemoSGD(FedSGDScheduler):

    def __init__(self,  # 没有 batch_size 和 merge_epochs 参数了
                 max_rounds: int = 0,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0,
                 learning_rate: float = 0.01,
                 momentum: float = 0.9) -> None:
        super().__init__(max_rounds=max_rounds,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds)
        self.learning_rate = learning_rate
        self.momentum = momentum

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.seed = 42
        torch.manual_seed(self.seed)

    # 加载训练集数据时要特殊处理，以保证 batch_size 等于训练样本总数，否则 FedSGDScheduler 会报错
    def build_train_dataloader(self) -> DataLoader:
        dataset = torchvision.datasets.MNIST(
            'data',  # 数据下载目录
            train=True,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        return DataLoader(dataset=dataset, batch_size=len(dataset), shuffle=True)

    """
    后面的部分与 DemoAvg 一模一样。
    """

    def build_model(self) -> nn.Module:  # 返回模型实例
        model = ConvNet()
        return model.to(self.device)

    def build_optimizer(self, model: nn.Module) -> optim.Optimizer:  # 返回优化器实例
        return optim.SGD(model.parameters(),
                         lr=self.learning_rate,
                         momentum=self.momentum)

    # 本示例不使用验证集数据，所以不需要实现 build_validation_dataloader

    # 网络下载 MNIST 测试数据用于测试，如果是实际使用场景，需要开发者自行处理数据加载
    def build_test_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=False
        )

    def train_an_epoch(self) -> None:  # 执行一个 epoch 训练的逻辑，与本地训练模型时的代码相同
        self.model.train()
        for data, labels in self.train_loader:
            data, labels = data.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.nll_loss(output, labels)
            loss.backward()
            self.optimizer.step()

    def run_test(self):  # 执行一次测试的逻辑，与本地模型测时的代码相同
        self.model.eval()
        start = time()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, labels in self.test_loader:
                data, labels = data.to(self.device), labels.to(self.device)
                output: torch.Tensor = self.model(data)
                test_loss += F.nll_loss(output, labels, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(labels.view_as(pred)).sum().item()

        end = time()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy

        # 记录 TensorBoard 日志，self.current_round 的值为“当前是训练的第几轮”
        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

使用 `FedSGDScheduler` 训练 `ConvNet` 的调度器设计完毕。 运行 `FedSGDScheduler` 与运行 `FedAvgScheduler` 完全一致，只是调度器实现类不同。此处不再赘述，刚兴趣的读者可自行尝试。

### `DPFedAvgScheduler` 介绍

`DPFedAvgScheduler` 也继承了 `FedAvgScheduler`，是其子类。`DPFedAvgScheduler` 与 `FedAvgScheduler` 的区别主要有两处：
1. 初始化参数不同，新增了 `w_cap`、`q`、`S`、`z` 四个初始化参数，各自代表[原始论文](https://arxiv.org/abs/1710.06963)中的同名超参数。
2. DPFedAvg 算法在本地完成每一个 batch 的训练之后，就需要全体协商计算 DP 噪音数值，因此不再使用训练接口 `train_an_epoch`，转而实现 `train_a_batch` 接口支持训练。

In [ ]:
@abstractmethod
def train_a_batch(self, *batch_train_data):
    """本地训练一个 batch 的代码逻辑，与本地训练模型时的代码相同。

    此接口必须实现。
    """

In [ ]:
from time import time

import torch
import torch.nn.functional as F
import torchvision
from cnn_net import ConvNet
from torch import nn, optim
from torch.utils.data import DataLoader

from alphafed.fed_avg import DPFedAvgScheduler


class DemoDP(DPFedAvgScheduler):

    def __init__(self,
                 w_cap: int,
                 q: float,
                 S: float,
                 z: float,
                 max_rounds: int = 0,
                 merge_epochs: int = 1,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0,
                 involve_aggregator: bool = False) -> None:
        super().__init__(w_cap=w_cap,
                         q=q,
                         S=S,
                         z=z,
                         max_rounds=max_rounds,
                         merge_epochs=merge_epochs,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds,
                         involve_aggregator=involve_aggregator)
        self.batch_size = 128
        self.learning_rate = 0.01
        self.momentum = 0.9
        self.random_seed = 42

        torch.manual_seed(self.random_seed)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def train_a_batch(self, *batch_train_data):
        data: torch.Tensor
        labels: torch.Tensor
        data, labels = batch_train_data
        data, labels = data.to(self.device), labels.to(self.device)
        self.optimizer.zero_grad()
        output = self.model(data)
        loss = F.nll_loss(output, labels)
        loss.backward()
        self.optimizer.step()

    """
    后面的部分与 DemoAvg 一致。
    """

    def build_model(self) -> nn.Module:  # 返回模型实例
        model = ConvNet()
        return model.to(self.device)

    def build_optimizer(self, model: nn.Module) -> optim.Optimizer:  # 返回优化器实例
        return optim.SGD(self.model.parameters(),
                         lr=self.learning_rate,
                         momentum=self.momentum)

    # 网络下载 MNIST 训练数据用于训练，如果是实际使用场景，需要开发者自行处理数据加载
    def build_train_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=True
        )

    # 本示例不使用验证集数据，所以不需要实现 build_validation_dataloader

    # 网络下载 MNIST 测试数据用于测试，如果是实际使用场景，需要开发者自行处理数据加载
    def build_test_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=False
        )

    def run_test(self):  # 执行一次测试的逻辑，与本地模型测时的代码相同
        self.model.eval()
        start = time()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, labels in self.test_loader:
                data, labels = data.to(self.device), labels.to(self.device)
                output: torch.Tensor = self.model(data)
                test_loss += F.nll_loss(output, labels, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(labels.view_as(pred)).sum().item()

        end = time()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy

        # 记录 TensorBoard 日志，self.current_round 的值为“当前是训练的第几轮”
        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

使用 `DPFedAvgScheduler` 训练 `ConvNet` 的调度器设计完毕。 运行 `DPFedAvgScheduler` 与运行 `FedAvgScheduler` 完全一致，只是调度器实现类不同。此处不再赘述，刚兴趣的读者可自行尝试。

### `SecureFedAvgScheduler` 介绍

`SecureFedAvgScheduler` 也继承了 `FedAvgScheduler`，是其子类。`SecureFedAvgScheduler` 与 `FedAvgScheduler` 相比仅有两个初始化参数不同，其它均一模一样。一是 `SecureFedAvgScheduler` 多了一个初始化参数 `t`，用来决定秘密分享时，恢复秘密最少需要几份秘密碎片。二是 SecureFedAvg 算法不允许聚合方参与本地训练，否则有数据泄露的风险，因此不支持 `involve_aggregator` 参数。

In [ ]:
from time import time

import torch
import torch.nn.functional as F
import torchvision
from cnn_net import ConvNet
from torch import nn, optim
from torch.utils.data import DataLoader

from alphafed.fed_avg import SecureFedAvgScheduler


class DemoSecure(SecureFedAvgScheduler):

    def __init__(self,
                 t: int,
                 max_rounds: int = 0,
                 merge_epochs: int = 1,
                 calculation_timeout: int = 300,
                 log_rounds: int = 0) -> None:
        super().__init__(t=t,
                         max_rounds=max_rounds,
                         merge_epochs=merge_epochs,
                         calculation_timeout=calculation_timeout,
                         log_rounds=log_rounds)
        self.batch_size = 128
        self.learning_rate = 0.01
        self.momentum = 0.9
        self.random_seed = 42

        torch.manual_seed(self.random_seed)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    """
    后面的部分与 DemoAvg 一模一样，可以直接跳过。
    """

    def build_model(self) -> nn.Module:  # 返回模型实例
        model = ConvNet()
        return model.to(self.device)

    def build_optimizer(self, model: nn.Module) -> optim.Optimizer:  # 返回优化器实例
        return optim.SGD(self.model.parameters(),
                         lr=self.learning_rate,
                         momentum=self.momentum)

    # 网络下载 MNIST 训练数据用于训练，如果是实际使用场景，需要开发者自行处理数据加载
    def build_train_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=True,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=True
        )

    # 本示例不使用验证集数据，所以不需要实现 build_validation_dataloader

    # 网络下载 MNIST 测试数据用于测试，如果是实际使用场景，需要开发者自行处理数据加载
    def build_test_dataloader(self) -> DataLoader:
        return DataLoader(
            torchvision.datasets.MNIST(
                'data',  # 数据下载目录
                train=False,
                download=True,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            ),
            batch_size=self.batch_size,
            shuffle=False
        )

    def train_an_epoch(self) -> None:  # 执行一个 epoch 训练的逻辑，与本地训练模型时的代码相同
        self.model.train()
        for data, labels in self.train_loader:
            data, labels = data.to(self.device), labels.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(data)
            loss = F.nll_loss(output, labels)
            loss.backward()
            self.optimizer.step()

    def run_test(self):  # 执行一次测试的逻辑，与本地模型测时的代码相同
        self.model.eval()
        start = time()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, labels in self.test_loader:
                data, labels = data.to(self.device), labels.to(self.device)
                output: torch.Tensor = self.model(data)
                test_loss += F.nll_loss(output, labels, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(labels.view_as(pred)).sum().item()

        end = time()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy

        # 记录 TensorBoard 日志，self.current_round 的值为“当前是训练的第几轮”
        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

使用 `SecureFedAvgScheduler` 训练 `ConvNet` 的调度器设计完毕。运行 `SecureFedAvgScheduler` 与运行 `FedAvgScheduler` 完全一致，只是调度器实现类不同。此处不再赘述，刚兴趣的读者可自行尝试。